# NavWareSet Quantitative Analysis

This notebook is a quantitative analysis of 545 hand selected robot-human tracks from the NavWareSet dataset. Almost all tracks from all scenarios where compliant and non-compliant behavior are present are analysed. All metrics were calculated for each possible combination of scenario, robot and behavior. Our objective is not to propose new metrics, but to test how our new dataset scores against metrics already proposed in the literature about robot social navigation. 

The chosen metrics were:
- Vavg (m/s) [1] - Average velocity of the robot across a number of tracks;
- Aavg (m/s2) [1] - Average aceleration of the robot across a number of tracks;
- Javg (m/s3) [1] - Average Jerk of the robot across a number of tracks;
- PLavg (m) [1] - Agerage Path Length across a number of tracks;
- PSC (%) [2] - Ratio of the trajectory with the minimum distance to a human under a given threshold. If the threshold is 0.5m, it is referred to as Personal Space Compliance (PSC);
- DHmin [3] - Average minimum distance to a human across a number of tracks;

[(1) - Arena-Bench: A Benchmarking Suite for Obstacle Avoidance Approaches in Highly Dynamic Environments](https://arxiv.org/pdf/2206.05728)

[(2) - iGibson 2.0: Object-Centric Simulation for Robot Learning of Everyday Household Tasks](https://arxiv.org/pdf/2108.03272)

[(3) - Principles and Guidelines for Evaluating Social Robot Navigation Algorithms](https://arxiv.org/pdf/2306.16740)

## Separating the scenes

In total, 5 scenes were analysed:
- Frontal Approach (fa);
- Pedestrian Obstruction (po);
- Blind Corner (bc);
- Perpendicular Crossing (pc);
- Circular Crossing (cc).

Two robots were used:
- Cleapath Jackal (j);
- Toyota HSR (h).

And two robot behaviors were observed:
- Compliant (c);
- Non-compliant (n).

In the cell below all scenes are divided acording to the classifications above.

In [23]:
# scenes corresponding to each scenario combination
fa_h_c = [1, 27]
fa_h_n = [8, 34]
fa_j_c = [15, 41]
fa_j_n = [21, 47]
po_h_c = [2, 28]
po_h_n = [9, 35]
po_j_c = [16, 42]
po_j_n = [22]
bc_h_c = [3, 29]
bc_h_n = [10, 36]
bc_j_c = [17, 43]
bc_j_n = [23, 49]
pc_h_c = [5, 31]
pc_h_n = [12, 38]
pc_j_c = [19, 45]
pc_j_n = [25, 51]
cc_h_c = [6, 32]
cc_h_n = [13, 39]
cc_j_c = [20, 46]
cc_j_n = [26, 52]

all_groups = [fa_h_c, fa_h_n, fa_j_c, fa_j_n,
              po_h_c, po_h_n, po_j_c, po_j_n,
              bc_h_c, bc_h_n, bc_j_c, bc_j_n,
              pc_h_c, pc_h_n, pc_j_c, pc_j_n,
              cc_h_c, cc_h_n, cc_j_c, cc_j_n]  

## Calculating the metrics

Down below we see the functions implemented to calculate each of the forementioned metrics

In [24]:
from pathlib import Path
import pandas as pd
import numpy as np

root = Path("selected_tracks")

### Average speed

In [25]:
def global_average_speed(scene_list: list) -> float:
    """
    Global average robot speed across ALL tracks in ALL scenes (m/s), computed as:
        (sum of distances over all tracks) / (sum of durations over all tracks)

    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    total_dist = 0.0
    total_time = 0.0

    for scene_id, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2).sum()
            dt = t[-1] - t[0]

            if dt > 0:
                total_dist += dist
                total_time += dt

    return total_dist / total_time if total_time > 0 else 0.0

print(global_average_speed(po_j_n))

0.666501013767755


### Average aceleration

In [26]:
def global_average_acceleration(scene_list: list) -> float:
    """
    Global average robot acceleration magnitude across ALL tracks in ALL scenes (m/s^2).

    For each track:
      1) compute instantaneous speed v_i = ds_i / dt_i
      2) compute instantaneous acceleration a_i = dv_i / dt_i
      3) take mean(|a_i|) over the track (time-weighted)
    Then average globally across all tracks (time-weighted).

    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    sum_a_dt = 0.0
    sum_dt = 0.0

    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dt = np.diff(t)
            dx = np.diff(x)
            dy = np.diff(y)

            valid = dt > 0
            dt = dt[valid]
            dx = dx[valid]
            dy = dy[valid]

            if len(dt) < 2:
                continue

            # speed per interval
            ds = np.sqrt(dx**2 + dy**2)
            v = ds / dt  # m/s

            # acceleration per interval between speeds
            dv = np.diff(v)
            dt2 = dt[1:]  # time between v samples
            valid2 = dt2 > 0

            a = np.abs(dv[valid2] / dt2[valid2])  # m/s^2

            if len(a) == 0:
                continue

            # time-weighted accumulation for global mean
            sum_a_dt += np.sum(a * dt2[valid2])
            sum_dt += np.sum(dt2[valid2])

    return sum_a_dt / sum_dt if sum_dt > 0 else 0.0

print(global_average_acceleration(po_j_n))

0.5595769171212424


### Average Jerk

In [27]:
def global_average_jerk(scene_list: list) -> float:
    """
    Global average robot jerk magnitude across ALL tracks in ALL scenes (m/s^3).

    For each track:
      1) v_i = ds_i / dt_i
      2) a_i = dv_i / dt_i
      3) j_i = da_i / dt_i
      4) average |j_i| using a time-weighted mean
    Then average globally across all tracks (time-weighted).

    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    sum_j_dt = 0.0
    sum_dt = 0.0

    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dt = np.diff(t)
            dx = np.diff(x)
            dy = np.diff(y)

            valid = dt > 0
            dt = dt[valid]
            dx = dx[valid]
            dy = dy[valid]

            if len(dt) < 3:
                continue

            # v_i defined over dt[i]
            ds = np.sqrt(dx**2 + dy**2)
            v = ds / dt  # m/s

            # a_i defined over dt[1:]
            dv = np.diff(v)
            dt_a = dt[1:]
            valid_a = dt_a > 0
            a = dv[valid_a] / dt_a[valid_a]  # signed accel

            if len(a) < 2:
                continue

            # j_i defined over dt_a[1:]
            da = np.diff(a)
            dt_j = dt_a[1:][valid_a[1:]]  # keep consistent indexing
            valid_j = dt_j > 0

            j = np.abs(da[valid_j] / dt_j[valid_j])  # m/s^3

            if len(j) == 0:
                continue

            # time-weighted global mean of |jerk|
            sum_j_dt += np.sum(j * dt_j[valid_j])
            sum_dt += np.sum(dt_j[valid_j])

    return sum_j_dt / sum_dt if sum_dt > 0 else 0.0

print(global_average_jerk(po_j_n))

8.443075226144948


### Average Path length

In [28]:
def global_average_path_length(scene_list: list) -> float:
    """
    Global average path length traveled by the robot across ALL tracks in ALL scenes (m).
    Computed as the mean of path lengths, where each track's path length is the sum of 
    incremental distances traveled.

    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    total_path_length = 0.0
    track_count = 0

    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            # Calculate path length as sum of incremental distances
            path_length = np.sqrt(np.diff(x)**2 + np.diff(y)**2).sum()

            total_path_length += path_length
            track_count += 1

    return total_path_length / track_count if track_count > 0 else 0.0

print(global_average_path_length(po_j_n))

7.882381868518138


### Personal Space compliance

In [29]:
def global_average_personal_space_compliance(scene_list: list, threshold: float = 0.5) -> float:
    """
    Global average Personal Space Compliance (PSC) across ALL tracks in ALL scenes (%).
    
    PSC is the percentage of timestamps where the minimum distance to any human 
    is below the given threshold.
    
    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...
    
    Args:
        scene_list: List of scene IDs
        threshold: Distance threshold in meters (default: 0.5m for personal space)
    
    Returns:
        Average PSC (%) across all tracks
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    total_psc = 0.0
    track_count = 0
    
    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)
            
            robot_x = df["robot_x"].to_numpy(dtype=np.float64)
            robot_y = df["robot_y"].to_numpy(dtype=np.float64)
            
            # Detect format and calculate minimum distances per timestamp
            if "x" in df.columns and "y" in df.columns and "column" in df.columns:
                # Format 1: column-based (x, y, column for each participant)
                # Group by timestamp and find minimum distance to any participant at each timestamp
                min_distances_per_timestamp = []
                
                for timestamp in df["timestamp"].unique():
                    timestamp_data = df[df["timestamp"] == timestamp]
                    min_dist_at_timestamp = np.inf
                    
                    for participant_id in timestamp_data["column"].unique():
                        part_data = timestamp_data[timestamp_data["column"] == participant_id]
                        part_x = part_data["x"].values[0]
                        part_y = part_data["y"].values[0]
                        robot_x_ts = part_data["robot_x"].values[0]
                        robot_y_ts = part_data["robot_y"].values[0]
                        
                        dist = np.sqrt((robot_x_ts - part_x)**2 + (robot_y_ts - part_y)**2)
                        min_dist_at_timestamp = min(min_dist_at_timestamp, dist)
                    
                    if np.isfinite(min_dist_at_timestamp):
                        min_distances_per_timestamp.append(min_dist_at_timestamp)
            
            else:
                # Format 2: xy_pairs (x1, y1, x2, y2, ...)
                # Each row is one timestamp, so we can calculate directly
                min_distances_per_timestamp = []
                
                # Extract all participant numbers
                participant_nums = set()
                for col in df.columns:
                    if col.startswith('x') and col[1:].isdigit():
                        participant_nums.add(col[1:])
                
                # For each row (timestamp)
                for idx in range(len(df)):
                    min_dist_at_timestamp = np.inf
                    
                    for participant_num in sorted(participant_nums):
                        x_col = f'x{participant_num}'
                        y_col = f'y{participant_num}'
                        
                        part_x = df.iloc[idx][x_col]
                        part_y = df.iloc[idx][y_col]
                        robot_x_ts = df.iloc[idx]["robot_x"]
                        robot_y_ts = df.iloc[idx]["robot_y"]
                        
                        # Only calculate if both coordinates are valid
                        if not (np.isnan(part_x) or np.isnan(part_y)):
                            dist = np.sqrt((robot_x_ts - part_x)**2 + (robot_y_ts - part_y)**2)
                            min_dist_at_timestamp = min(min_dist_at_timestamp, dist)
                    
                    if np.isfinite(min_dist_at_timestamp):
                        min_distances_per_timestamp.append(min_dist_at_timestamp)
            
            # Calculate PSC: ratio of timestamps below threshold
            if len(min_distances_per_timestamp) > 0:
                min_distances_array = np.array(min_distances_per_timestamp)
                violations = np.sum(min_distances_array < threshold)
                psc = (violations / len(min_distances_per_timestamp)) * 100.0
            else:
                psc = 0.0
            
            total_psc += psc
            track_count += 1
    
    return total_psc / track_count if track_count > 0 else 0.0

print(global_average_personal_space_compliance(po_j_n))

0.4612159329140461


### Average minimal distance to a human

In [48]:
def global_average_minimal_distance(scene_list: list) -> float:
    """
    Global average minimal distance between robot and any human across ALL tracks in ALL scenes (m).

    For each track:
      1) compute the minimum distance to any human across all timestamps
    Then average all minimum distances across all tracks.

    Handles multiple humans per track. Finds the closest approach distance for each track,
    then averages those values.

    Handles both CSV formats:
    1. timestamp,x,y,column,robot_x,robot_y,robot_yaw_rad
    2. timestamp,robot_x,robot_y,robot_yaw_rad,x1,y1,x2,y2,...

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    min_distances_list = []
    
    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)
            
            robot_x = df["robot_x"].to_numpy(dtype=np.float64)
            robot_y = df["robot_y"].to_numpy(dtype=np.float64)
            
            # Initialize minimum distance for this track as infinity
            min_distance_track = np.inf
            
            # Detect format and calculate minimum distances to any human
            if "x" in df.columns and "y" in df.columns and "column" in df.columns:
                # Format 1: column-based (x, y, column for each participant)
                for participant_id in df["column"].unique():
                    participant_data = df[df["column"] == participant_id]
                    part_x = participant_data["x"].to_numpy(dtype=np.float64)
                    part_y = participant_data["y"].to_numpy(dtype=np.float64)
                    part_robot_x = participant_data["robot_x"].to_numpy(dtype=np.float64)
                    part_robot_y = participant_data["robot_y"].to_numpy(dtype=np.float64)
                    
                    distances = np.sqrt((part_robot_x - part_x)**2 + (part_robot_y - part_y)**2)
                    track_min = np.min(distances)
                    min_distance_track = min(min_distance_track, track_min)
            
            else:
                # Format 2: xy_pairs (x1, y1, x2, y2, ...)
                # Extract all participant numbers
                participant_nums = set()
                for col in df.columns:
                    if col.startswith('x') and col[1:].isdigit():
                        participant_nums.add(col[1:])
                
                # For each participant, calculate minimum distance
                for participant_num in sorted(participant_nums):
                    x_col = f'x{participant_num}'
                    y_col = f'y{participant_num}'
                    
                    if x_col in df.columns and y_col in df.columns:
                        part_x = df[x_col].to_numpy(dtype=np.float64)
                        part_y = df[y_col].to_numpy(dtype=np.float64)
                        
                        # Only calculate distances where both coordinates are valid
                        valid = ~(np.isnan(part_x) | np.isnan(part_y))
                        
                        if valid.any():
                            distances = np.sqrt((robot_x[valid] - part_x[valid])**2 + 
                                              (robot_y[valid] - part_y[valid])**2)
                            track_min = np.min(distances)
                            min_distance_track = min(min_distance_track, track_min)
            
            # Add this track's minimum distance if valid
            if np.isfinite(min_distance_track):
                min_distances_list.append(min_distance_track)
    
    # Return average of all minimum distances
    if len(min_distances_list) > 0:
        return np.mean(min_distances_list)
    else:
        return 0.0


## Making analysis

In [49]:
# Build and fill the results table
results = []

# Define all scenario combinations
scenarios = [
    ("Frontal Approach", "fa", [
        ("HSR", "Compliant", fa_h_c),
        ("HSR", "Non-compliant", fa_h_n),
        ("Jackal", "Compliant", fa_j_c),
        ("Jackal", "Non-compliant", fa_j_n),
    ]),
    ("Pedestrian Obstruction", "po", [
        ("HSR", "Compliant", po_h_c),
        ("HSR", "Non-compliant", po_h_n),
        ("Jackal", "Compliant", po_j_c),
        ("Jackal", "Non-compliant", po_j_n),
    ]),
    ("Blind Corner", "bc", [
        ("HSR", "Compliant", bc_h_c),
        ("HSR", "Non-compliant", bc_h_n),
        ("Jackal", "Compliant", bc_j_c),
        ("Jackal", "Non-compliant", bc_j_n),
    ]),
    ("Perpendicular Crossing", "pc", [
        ("HSR", "Compliant", pc_h_c),
        ("HSR", "Non-compliant", pc_h_n),
        ("Jackal", "Compliant", pc_j_c),
        ("Jackal", "Non-compliant", pc_j_n),
    ]),
    ("Circular Crossing", "cc", [
        ("HSR", "Compliant", cc_h_c),
        ("HSR", "Non-compliant", cc_h_n),
        ("Jackal", "Compliant", cc_j_c),
        ("Jackal", "Non-compliant", cc_j_n),
    ]),
]

# Calculate metrics for each combination
for scenario_name, scenario_short, combinations in scenarios:
    for robot, behavior, scene_list in combinations:
        vavg = global_average_speed(scene_list)
        aavg = global_average_acceleration(scene_list)
        javg = global_average_jerk(scene_list)
        plavg = global_average_path_length(scene_list)
        psc = global_average_personal_space_compliance(scene_list, threshold=0.5)
        dhmin = global_average_minimal_distance(scene_list)
        
        results.append({
            "Scenario": scenario_name,
            "Robot": robot,
            "Robot Behavior": behavior,
            "Vavg[1]": vavg,
            "Aavg[1]": aavg,
            "Javg[1]": javg,
            "PLavg (avg) [1]": plavg,
            "PSC [2]": psc,
            "DHmin avg [3]": dhmin,
        })

# Create DataFrame and display
results_df = pd.DataFrame(results)
results_df

,Scenario,Robot,Robot Behavior,Vavg[1],Aavg[1],Javg[1],PLavg (avg) [1],PSC [2],DHmin avg [3]
0,Frontal Approach,HSR,Compliant,0.296260,0.706948,11.734697,6.276202,0.492042,0.556392
1,Frontal Approach,HSR,Non-compliant,0.306745,0.735720,12.438214,6.196206,0.953583,0.503470
2,Frontal Approach,Jackal,Compliant,0.850625,0.710786,10.717854,8.173447,0.000000,1.056478
3,Frontal Approach,Jackal,Non-compliant,0.838914,0.716671,10.622924,7.623886,0.786786,0.561888
4,Pedestrian Obstruction,HSR,Compliant,0.292179,0.707200,11.820751,6.708710,0.000000,0.702893
5,Pedestrian Obstruction,HSR,Non-compliant,0.305779,0.722194,12.157474,6.208798,0.000000,0.672517
6,Pedestrian Obstruction,Jackal,Compliant,0.699302,0.603358,9.548696,8.321612,0.057143,0.776724
7,Pedestrian Obstruction,Jackal,Non-compliant,0.666501,0.559577,8.443075,7.882382,0.461216,0.704897
8,Blind Corner,HSR,Compliant,0.257968,0.700408,11.764283,5.448501,0.569689,0.622294
9,Blind Corner,HSR,Non-compliant,0.289296,0.754758,12.676405,5.253311,0.392795,0.626549


In [50]:
# Save results to CSV
output_file = "NavWareSet_Quantitative_Analysis_Results.csv"
results_df.to_csv(output_file, index=False)
print(f"Results saved to: {output_file}")
print(f"\nDataFrame shape: {results_df.shape}")
print(f"Columns: {list(results_df.columns)}")

Results saved to: NavWareSet_Quantitative_Analysis_Results.csv

DataFrame shape: (20, 9)
Columns: ['Scenario', 'Robot', 'Robot Behavior', 'Vavg[1]', 'Aavg[1]', 'Javg[1]', 'PLavg (avg) [1]', 'PSC [2]', 'DHmin avg [3]']
